### JSON files management
---
<b>METIS, Éveha International</b><br>
Author : Thomas Aubertier (thomas.aubertier@etu.sorbonne-universite.fr)
---

This notebook describe how to save and use data related to devices.

### ``0`` Import

Importing required libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import geophpy as geo
import geophpy.emi as gemi
import geophpy.visualization as gvis

### ``1`` Create and add device

Devices are dictionaries containing relevant data around the prospection.

They are not directly referencing a specific device, but the configuration used at the time of the prospection. Adding a GPS antenna or changing coil positions makes a new configuration, then should correspond to a new device entry.

Creating a new device configuration can be done using ``geophpy.add_device`` :
* (app_name) Device name (not used in processes)
* (config) Coil configuration (HCP, VCP, PAR... ,see documentation)
* (nb_channels) Number of coils
* (freq_list) Frequence list (in coil order, else add one value)
* ``gps`` If got gps data (default True
* ``gps_dec`` Shift between the GPS antenna and the device center, on both axis (should be ``[0,0]`` if none, by default), in meters
* ``TR_l`` Distance between each coil and the transmitter coil, on lateral axis (by default 0), in meters
* ``TR_t``Distance between each coil and the transmitter coil, on transversal axis (by default 0), in meters
* ``height`` Height of the device during the prospection (default 0.1), in meters
* ``bucking_coil`` Index of the bucking coil between coils (from 1 to the number of coils), if none set to 0 (default)
* ``coeff_c_ph`` Device constant given by the maker (in phase, by default is set to an array of 1s) ; can be computed from ball calibration with ``geophpy.emi.ball_calibr``
* ``coeff_c_qu`` Device constant given by the maker (in quadrature, by default is set to an array of 1s)
* ``config_angles`` If ``config = "CUS"`` (custom), define the angles of each coil (not implemented in most procedures)
* ``save`` Save configuration in ``geophpy`` (default True)
* ``error_code`` Instead of returning the dictionary, return an error code as an int (default False)

Output :
* Device's configuration dictionary OR 1 if already added, else 0

In [2]:
app_data = gemi.add_device('mini3L','HCP',3,[30000],gps=True,gps_dec=[0.25,-0.2],
                           TR_l=[0.32,0.71,1.18],TR_t=[0,0,0],height=0.1,bucking_coil=0,
                           coeff_c_ph=[0.5982,0.9839,0.7074],coeff_c_qu=[0.00591,0.0281,0.0745],save=True)

In [3]:
print(app_data)

{'app_id': 0, 'app_name': 'mini3L', 'config': 'HCP', 'GPS': True, 'GPS_dec': [0.25, -0.2], 'nb_paths': 3, 'TR_l': [0.32, 0.71, 1.18], 'TR_t': [0, 0, 0], 'TR': [np.float64(0.32), np.float64(0.71), np.float64(1.18)], 'height': 0.1, 'freq_list': [30000], 'bucking_coil': 0, 'coeff_c_ph': [0.5982, 0.9839, 0.7074], 'coeff_c_qu': [0.00591, 0.0281, 0.0745]}


### ``2`` See saved devices list

Each saved device is given an ``app_id`` which is a unique identifier used in functions.

To get the correspondance between ``app_id`` and the wanted configuration, one can use ``geophpy.emi.print_devices`` :
* ``uid`` Device's ``"app_id"`` value (by default, prints all)

In [4]:
e = gemi.add_device('mini3L','VCP',3,[30000],gps=True,gps_dec=[0.25,-0.2],
                           TR_l=[0.32,0.71,1.18],TR_t=[0,0,0],height=0.1,bucking_coil=0,
                           coeff_c_ph=[0.8471,0.8585,0.4039],coeff_c_qu=[0.00599,0.0290,0.0785],save=True,error_code=True)

In [5]:
gemi.print_devices()


--------------------------------------------------
0 : mini3L (HCP)
	GPS : True
	Nb T/R : 3, 
	Pos l : [0.32, 0.71, 1.18], Pos t : [0, 0, 0]
	z : 0.1, Frequences : [30000]
--------------------------------------------------
1 : mini3L (VCP)
	GPS : True
	Nb T/R : 3, 
	Pos l : [0.32, 0.71, 1.18], Pos t : [0, 0, 0]
	z : 0.1, Frequences : [30000]
--------------------------------------------------



In [6]:
gemi.print_devices(0)


--------------------------------------------------
0 : mini3L (HCP)
	GPS : True
	Nb T/R : 3, 
	Pos l : [0.32, 0.71, 1.18], Pos t : [0, 0, 0]
	z : 0.1, Frequences : [30000]
--------------------------------------------------



### ``3`` Modify device

This operation will keep the current order (``app_id``).

``geophpy.emi.modify_device`` :
* Device's ``"app_id"`` value or loaded device
* Dictionary containing all keys to overwrite

Output :
* Updated device's configuration dictionary

*Note : modifying a loaded device will not change any saved configuration.*

In [7]:
app_data = gemi.modify_device(0,{"app_name" : ";)"})

--------------------------------------------------
0 : mini3L (HCP)
	GPS : True
	Nb T/R : 3, 
	Pos l : [0.32, 0.71, 1.18], Pos t : [0, 0, 0]
	z : 0.1, Frequences : [30000]
--------------------------------------------------



This function prints the old values (for verification purposes).

We can then check that the changes have indeed been applied.

In [8]:
gemi.print_devices()


--------------------------------------------------
0 : ;) (HCP)
	GPS : True
	Nb T/R : 3, 
	Pos l : [0.32, 0.71, 1.18], Pos t : [0, 0, 0]
	z : 0.1, Frequences : [30000]
--------------------------------------------------
1 : mini3L (VCP)
	GPS : True
	Nb T/R : 3, 
	Pos l : [0.32, 0.71, 1.18], Pos t : [0, 0, 0]
	z : 0.1, Frequences : [30000]
--------------------------------------------------



In [10]:
print(app_data["app_name"])

;)


### ``4`` Delete device

This operation will shift the current order (``app_id``) by one for every configuration that come after.

``geophpy.emi.remove_device`` :
* Device's ``"app_id"`` value

In [11]:
gemi.remove_device(0)

In [12]:
gemi.print_devices()


--------------------------------------------------
0 : mini3L (VCP)
	GPS : True
	Nb T/R : 3, 
	Pos l : [0.32, 0.71, 1.18], Pos t : [0, 0, 0]
	z : 0.1, Frequences : [30000]
--------------------------------------------------



### ``5`` Load device from list

``geophpy.emi.find_device`` :
* Device's ``"app_id"`` value

Output :
* Device's configuration dictionary

In [13]:
app_data = gemi.find_device(0)

--------------------------------------------------
0 : mini3L (VCP)
	GPS : True
	Nb T/R : 3, 
	Pos l : [0.32, 0.71, 1.18], Pos t : [0, 0, 0]
	z : 0.1, Frequences : [30000]
--------------------------------------------------



In [14]:
print(app_data)

{'app_id': 0, 'app_name': 'mini3L', 'config': 'VCP', 'GPS': True, 'GPS_dec': [0.25, -0.2], 'nb_paths': 3, 'TR_l': [0.32, 0.71, 1.18], 'TR_t': [0, 0, 0], 'TR': [0.32, 0.71, 1.18], 'height': 0.1, 'freq_list': [30000], 'bucking_coil': 0, 'coeff_c_ph': [0.8471, 0.8585, 0.4039], 'coeff_c_qu': [0.00599, 0.029, 0.0785]}
